In [1]:
import pandas as pd

## Notebook for making predictions with different models etc. 


### Reading dataset stored in other notebooks

In [2]:

%store -r data_A
%store -r data_B
%store -r data_C

data_A = data_A
data_B = data_B
data_C = data_C

%store -r X_test_estimated_a_corr 
%store -r X_test_estimated_b_corr 
%store -r X_test_estimated_c_corr

X_test_estimated_a_corr = X_test_estimated_a_corr 
X_test_estimated_b_corr = X_test_estimated_b_corr
X_test_estimated_c_corr = X_test_estimated_c_corr

print(X_test_estimated_a_corr.shape)


(720, 44)


### Preparing training and testing data to train on

- we split the training sets into train and test data. In this way we can test our model.


In [3]:
import sklearn.model_selection as ms

#preparing data
X_A = data_A.iloc[:,2:-1] #independent columns
y_A = data_A.iloc[:,0]   #target column i.e uv stråling 

X_B = data_B.iloc[:,2:-1] #independent columns
y_B = data_B.iloc[:,0]    #target column i.e uv stråling 

X_C = data_C.iloc[:,2:-1] #independent columns
y_C = data_C.iloc[:,0]   #target column i.e uv stråling 


X_A_train, X_A_test, y_A_train, y_A_test = ms.train_test_split(X_A, y_A)
X_B_train, X_B_test, y_B_train, y_B_test = ms.train_test_split(X_B, y_B)
X_C_train, X_C_test, y_C_train, y_C_test = ms.train_test_split(X_C, y_C)

print(X_A_train.shape)
print(X_B_train.shape)
print(X_C_train.shape)


(25563, 42)
(24636, 24)
(24116, 24)


In [4]:
# fixing test data

X_test_A = X_test_estimated_a_corr[X_A_train.columns.intersection(X_test_estimated_a_corr.columns)]
X_test_B = X_test_estimated_b_corr[X_A_train.columns.intersection(X_test_estimated_b_corr.columns)]
X_test_C = X_test_estimated_c_corr[X_A_train.columns.intersection(X_test_estimated_c_corr.columns)]


### Preds using GradientBoostingRegressor



In [9]:
from sklearn.ensemble import GradientBoostingRegressor

## defining model and hyper-parameters
model_A = GradientBoostingRegressor(learning_rate=0.5, n_estimators=150)
model_B = GradientBoostingRegressor(learning_rate=0.5, n_estimators=150)
model_C = GradientBoostingRegressor(learning_rate=0.5, n_estimators=150)

model = GradientBoostingRegressor(learning_rate=0.5, n_estimators=150)


In [10]:
# training model 
model_A.fit(X_A_train, y_A_train)
model_B.fit(X_B_train, y_B_train)
model_C.fit(X_C_train, y_C_train)

model.fit(X_B_train, y_B_train)
model.fit(X_C_train, y_C_train)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/valid

GradientBoostingRegressor(learning_rate=0.5, n_estimators=150)

In [21]:
# testing model 

score_A = model_A.score(X_A_test, y_A_test)
score_B = model_B.score(X_B_test, y_B_test)
score_C = model_C.score(X_A_test, y_A_test)

score_mA = model.score(X_A_test, y_A_test)
score_mB = model.score(X_B_test, y_B_test)
score_mC = model.score(X_C_test, y_C_test)

print(score_A)
print(score_B)
print(score_C)

print(score_mA)
print(score_mB)
print(score_mC)

print(model_A.feature_importances_)


0.8931301256486577
0.8646083317680454
-0.0487743348066334
-0.04801447584988727
0.5911103508334048
0.8417045073963214
[6.77029007e-03 5.40962375e-03 0.00000000e+00 7.79809346e-03
 0.00000000e+00 6.11552230e-04 1.10230280e-04 2.36719380e-03
 5.55077961e-03 2.40265778e-03 0.00000000e+00 6.04569861e-05
 2.85466895e-03 2.58917830e-03 5.38821567e-05 1.12350341e-03
 1.17896048e-02 5.40082248e-02 1.75596071e-02 4.53833946e-03
 0.00000000e+00 1.03231858e-02 7.27534869e-03 8.56803579e-01]


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/valid

## Random forest regressor 

In [34]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

rfmA = RandomForestRegressor(max_depth=6, max_features=None, max_leaf_nodes=10)
rfmB = RandomForestRegressor(max_depth=6, max_features=None, max_leaf_nodes=10, n_estimators=50)
rfmC = RandomForestRegressor(max_depth=6, max_features=None, max_leaf_nodes=10, n_estimators=25)

rfmA = RandomForestRegressor(max_leaf_nodes=10)
rfmB = RandomForestRegressor(max_leaf_nodes=10)
rfmC = RandomForestRegressor(max_leaf_nodes=10)

rfm = RandomForestRegressor(max_depth=10)

### Finding best hyperparameters 
Below code takes around 1 hour to run

Runing the below code found that
- max depth = 6 
- max features = none 
- max leaf nodes = 9 (propably better with higher)
- for A n estimators = default 
- for B n estimators = 50 
- for C n estimators = 25 

In [532]:
param_grid = {
    'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}

grid_search_A = GridSearchCV(RandomForestRegressor(), param_grid=param_grid)
grid_search_A.fit(X_A_train, y_A_train)

grid_search_B = GridSearchCV(RandomForestRegressor(), param_grid=param_grid)
grid_search_B.fit(X_B_train, y_B_train)

grid_search_C = GridSearchCV(RandomForestRegressor(), param_grid=param_grid)
grid_search_C.fit(X_C_train, y_C_train)

print("A: " , grid_search_A.best_estimator_)

print("B: ", grid_search_B.best_estimator_)

print("C: ", grid_search_C.best_estimator_)



A:  RandomForestRegressor(max_depth=6, max_features=None, max_leaf_nodes=9)
B:  RandomForestRegressor(max_depth=6, max_features=None, max_leaf_nodes=9,
                      n_estimators=50)
C:  RandomForestRegressor(max_depth=6, max_features=None, max_leaf_nodes=9,
                      n_estimators=25)


In [35]:
rfmA.fit(X_A_train, y_A_train)
rfmB.fit(X_B_train, y_B_train)
rfmC.fit(X_C_train, y_C_train)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/valid

RandomForestRegressor(max_leaf_nodes=10)

In [36]:

score_A = rfmA.score(X_A_test, y_A_test)
score_B = rfmB.score(X_B_test, y_B_test)
score_C = rfmC.score(X_C_test, y_C_test)


print(score_A)
print(score_B)
print(score_C)



0.8014643851659848
0.7521640933319699
0.6763562667569902


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/valid

## Cat boost 

160

In [5]:
from catboost import CatBoostRegressor

catModel_A = CatBoostRegressor()
catModel_B = CatBoostRegressor()
catModel_C = CatBoostRegressor()

catModel = CatBoostRegressor()


In [6]:
catModel_A.fit(X_A_train, y_A_train)
catModel_B.fit(X_B_train, y_B_train)
catModel_C.fit(X_C_train, y_C_train)

# catModel.fit(X_A_train, y_A_train)
# catModel.fit(X_B_train, y_B_train)
# catModel.fit(X_C_train, y_C_train)


Learning rate set to 0.068325
0:	learn: 1105.1632226	total: 57.9ms	remaining: 57.8s
1:	learn: 1046.5346610	total: 61.4ms	remaining: 30.6s
2:	learn: 993.4801303	total: 64.3ms	remaining: 21.4s
3:	learn: 944.6726448	total: 67.8ms	remaining: 16.9s
4:	learn: 899.3964942	total: 71.2ms	remaining: 14.2s
5:	learn: 858.8513636	total: 74.4ms	remaining: 12.3s
6:	learn: 820.6609287	total: 77.3ms	remaining: 11s
7:	learn: 786.3596108	total: 80.2ms	remaining: 9.95s
8:	learn: 753.6544454	total: 83.1ms	remaining: 9.15s
9:	learn: 724.2470895	total: 85.9ms	remaining: 8.51s
10:	learn: 697.8725175	total: 88.7ms	remaining: 7.97s
11:	learn: 672.8756327	total: 91.6ms	remaining: 7.54s
12:	learn: 650.8006799	total: 94.5ms	remaining: 7.17s
13:	learn: 629.8569156	total: 97.2ms	remaining: 6.85s
14:	learn: 611.8751427	total: 99.8ms	remaining: 6.56s
15:	learn: 595.3959370	total: 103ms	remaining: 6.31s
16:	learn: 580.5612297	total: 106ms	remaining: 6.11s
17:	learn: 566.8409497	total: 108ms	remaining: 5.92s
18:	learn: 

In [8]:
score_A = catModel_A.score(X_A_test, y_A_test)
score_B = catModel_B.score(X_B_test, y_B_test)
score_C = catModel_C.score(X_C_test, y_C_test)

print(score_A)
print(score_B)
print(score_C)


0.8778693672560491
0.8609178175125225
0.8359325107341539


## Stacking

198

In [45]:
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor

estimators = [
    ("ada", AdaBoostRegressor()),
    ("gbr", GradientBoostingRegressor())
]

reg_A = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor()
)

reg_B = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor()
)

reg_C = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor()
)

reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor()
)

In [46]:
reg_A.fit(X_A_train, y_A_train)
reg_B.fit(X_B_train, y_B_train)
reg_C.fit(X_C_train, y_C_train)

reg.fit(X_A_train, y_A_train)
reg.fit(X_B_train, y_B_train)
reg.fit(X_C_train, y_C_train)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/valid

StackingRegressor(estimators=[('ada', AdaBoostRegressor()),
                              ('gbr', GradientBoostingRegressor())],
                  final_estimator=RandomForestRegressor())

In [47]:
score_A = reg_A.score(X_A_test, y_A_test)
score_B = reg_B.score(X_B_test, y_B_test)
score_C = reg_C.score(X_C_test, y_C_test)

score_mA = reg.score(X_A_test, y_A_test)
score_mB = reg.score(X_B_test, y_B_test)
score_mC = reg.score(X_C_test, y_C_test)

print(score_A)
print(score_B)
print(score_C)

print(score_mA)
print(score_mB)
print(score_mC)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/valid

0.8307553075732385
0.7698363559481678
0.6907999679790471
-0.05837661460861154
0.6060524426525795
0.68536986221043


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


# Create subission

## NB! THE RESULTING CSV FILE IS MISSING THE "id" LABLE FOR ITS FIRST COLUMN; QUICK FIX: JUST GO INTO THE CSV IN VS CODE AND ADD MANUALLY AT THE TOP

In [9]:
## Specify the model to be used
pred_A = catModel_A.predict(X_test_A)
pred_B = catModel_B.predict(X_test_B)
pred_C = catModel_C.predict(X_test_C)


## drit i å forstå, det funker 
test = pd.read_csv('test.csv')

df_A = pd.DataFrame()

df_A["prediction"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["prediction"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["prediction"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df_C['prediction'] = df_C['prediction'].astype(float)

test = test[test.columns.intersection(["id", "time", "location"])]

df = df.drop("location", axis=1)

df.set_index(test["id"])
# df["id"] = test["id"]

# df = df[["id", "prediction"]]

df[df < 0 ] = 0

# NAME THE FILE 
df.to_csv("sub54.csv")

df


CatBoostError: /Users/zomb-ml-platform-msk/go-agent-21.2.0/pipelines/BuildMaster/catboost.git/catboost/libs/data/model_dataset_compatibility.cpp:81: At position 1 should be feature with name ceiling_height_agl:m (found air_density_2m:kgm3).